In [9]:
from dotenv import load_dotenv

from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate

from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

from readability import Readability
from readability.exceptions import ReadabilityException

load_dotenv()

llm = HuggingFaceEndpoint(repo_id="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


ValidationError: 1 validation error for HuggingFaceEndpoint
__root__
  Could not authenticate with huggingface_hub. Please check your API token. (type=value_error)

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools, AgentType, initialize_agent

## Serpapi

In [ ]:
tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.invoke("How much did the Tesla stock price change recently?")

c:\Users\alanj\Documents\GitHub\dynolearn\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO (Request ID: 6ciGvEXRFz4UbA0lV6vpT)

Model is overloaded

## Tavily

In [15]:
# from langchain_community.tools.tavily_search import TavilySearchResults

# tools = [TavilySearchResults(max_results=5)]

# prompt = hub.pull("hwchase17/react")

tasks_template = """<|im_start|>system
As a seasoned game designer with expertise in creating engaging educational content for children, 

You are to create a list of tasks as a JSON list, with a clear description for each task that will motivate and engage young learners. 
Your tasks should be designed to gradually increase in difficulty, with a mix of fun and challenging activities that cater to different learning styles.
A childs reading level is a measure of how easy a text is to read. It is usually measured in years, so a reading level of 5 means that a 5 year old can read it. 
If a child is struggling with reading, they may have a reading level that is lower than their actual age.

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

NOTE: {format_instructions}

<|im_end|>

<|im_start|>user
Question: {name} is {age} years old and has a reading level age of {reading_level}. They need help with learning {teaching_task}. Please make a list of 10 suitable game scene tasks and prompts to learn {teaching_task}.
<|im_end|>

<|im_start|>assistant

Thought:{agent_scratchpad}
"""

class Tasks(BaseModel):
    tasks: List = Field(description="list of game scene tasks")


parser = JsonOutputParser(pydantic_object=Tasks)

tasks_prompt_json = PromptTemplate(
    template=tasks_template,
    input_variables=["name", "age", "reading_level", "teaching_task"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

print(parser.get_format_instructions())

agent = create_react_agent(llm=llm, tools=tools, prompt=tasks_prompt_json)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke({
        "name": "Sophie",
        "age": 7,
        "reading_level": 5,
        "teaching_task": "How to deal with strong smells"
    })


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"tasks": {"title": "Tasks", "description": "list of game scene tasks", "type": "array", "items": {}}}, "required": ["tasks"]}
```


> Entering new AgentExecutor chain...


OverloadedError: Model is overloaded

In [ ]:
from transformers import load_tool

tool = load_tool("text-to-speech")
audio = tool("This is a text to speech tool")

AttributeError: module 'requests' has no attribute 'exceptions'